In [20]:
# utils/llm.py

import os

from dotenv import load_dotenv
load_dotenv()
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_pinecone import PineconeVectorStore

text = '''
    넌 질문-답변을 도와주는 AI 영화 추천기야.
    아래 제공되는 Context를 통해서 사용자 Question에 대해 답을 해줘야해.

    Context에는 직접적으로 없어도, 추론하거나 계산할 수 있는 답변은 최대한 만들어 봐.
    해당 영화가 존재한다면, 답변 마지막에 "해당 영화 및 사건과 관련된 정보를 POV Timeline에서 확인하실 수 있습니다."
    만약 우리가 제공한 csv 파일에 관련 영화가 존재하지 않는다면, "죄송합니다. 관련 영화가 없습니다. 다른 영화를 추천 받으시겠어요?"라는 답변을 생성해줘.

    답은 적절히 \n를 통해 문단을 나눠줘 한국어로 만들어 줘. 
    # Question:
    {question}

    # Context:
    {context}


    # Answer:
    '''
    
vectorstore = PineconeVectorStore.from_existing_index(
    index_name = os.environ.get('INDEX_NAME'),
    embedding=OpenAIEmbeddings()
)

# 5. Retrieve
retriever = vectorstore.as_retriever()


# 6. Prompting
prompt = PromptTemplate.from_template(text)
    
def query_llm(user_input):
    # 7. LLM
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    parser = StrOutputParser()

    # 8. Chain
    chain = (
        {'context': retriever, 'question': RunnablePassthrough()}
        | prompt
        | llm
        | parser
    )

    ans = chain.invoke(user_input)
    return ans

In [21]:
from pydantic import BaseModel, Field

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

# LLM with function call
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

In [23]:
# 테스트용 코드
retrieval_grader = grade_prompt | structured_llm_grader
question = "80년대 배경의 전두환 등장 영화는?"  # Yes
docs = retriever.invoke(question)

In [24]:
print(docs)

[Document(id='dda6abfe-278f-44c9-bada-7fd4412b8bcb', metadata={'row': 29.0, 'source': './database.csv'}, page_content='id: 30\nhistory year: 1980\nhistory description: 80년대, 5.18민주화운동, 계엄군, 전두환, 광주, 학살\ngenre: \nmovie description: 김상경, 이요원\n저항, 슬픔, 비극\nmovie title: 화려한 휴가\nrelease date: 2007.07.25'), Document(id='3ab2cc02-2a9b-47bd-be06-98ae988d8595', metadata={'row': 29.0, 'source': './database.csv'}, page_content='id: 30\nhistory year: 1980\nhistory description: 80년대, 5.18민주화운동, 계엄군, 전두환, 광주, 학살\ngenre: \nmovie description: 김상경, 이요원\n저항, 슬픔, 비극\nmovie title: 화려한 휴가\nrelease date: 2007.07.25'), Document(id='9322a0df-5359-4da7-a40e-2d80ffb5fd2a', metadata={'row': 19.0, 'source': './database.csv'}, page_content='id: 20\nhistory year: 1964\nhistory description: 1960년 3.15 부정선거, 4.19혁명\n1961년 5.16군사정변\n1968년 김신조 사건\n60년대\ngenre: \nmovie description: 송강호, 대통령(독재자)의 이발사\n역사 속의 개인\nmovie title: 효자동 이발사\nrelease date: 2004.05.05'), Document(id='47bacd9e-ff93-4191-bed8-0d48d691c3e6', metadata=

In [25]:
print(docs[2])

page_content='id: 20
history year: 1964
history description: 1960년 3.15 부정선거, 4.19혁명
1961년 5.16군사정변
1968년 김신조 사건
60년대
genre: 
movie description: 송강호, 대통령(독재자)의 이발사
역사 속의 개인
movie title: 효자동 이발사
release date: 2004.05.05' metadata={'row': 19.0, 'source': './database.csv'}


In [26]:
doc_txt = docs[2].page_content

In [27]:
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='no'


In [28]:
print(retrieval_grader.invoke({"question": question, "document": docs}))

# 241017 16:03

binary_score='yes'


In [29]:
### Generate

from langchain import hub # 좋은 프롬프트들을 가져와서 뜨게
from langchain_core.output_parsers import StrOutputParser

# Prompt
# prompt = hub.pull("rlm/rag-prompt") # 이게 가져오는 프롬프트 이름임

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

80년대 배경의 전두환이 등장하는 영화로는 "화려한 휴가"가 있습니다. 이 영화는 1980년대의 5.18 민주화운동과 관련된 사건들을 다루고 있으며, 전두환과 계엄군의 역할이 중요한 배경으로 설정되어 있습니다. 

영화는 김상경과 이요원이 출연하며, 저항과 슬픔, 비극을 주제로 하고 있습니다. "화려한 휴가"는 2007년 7월 25일에 개봉되었습니다.

해당 영화 및 사건과 관련된 정보를 POV Timeline에서 확인하실 수 있습니다.


In [30]:
### Question Re-writer

# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n 
     for web search. Look at the input and try to reason about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

question = "야구 역대 최고 관중 수는 얼마인가요?" # Yes

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'역대 야구 경기 중 최고 관중 수는 얼마인가요?'

In [31]:
### Search

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(max_results=3)

web_search_tool.invoke('정마담이 빌린 돈은?')

# 24-10-21 11:12

[{'url': 'https://www.kmib.co.kr/article/view.asp?arcid=0013476407',
  'content': "정마담이 폭로 가담한 이유 입력: 2019-07-09 11:24. 수정: ... (돈을 빌린 사람)들이 금리 인하를 체감하기까지 상당한 시간이 필요할 전망이다. 시장이 기준금리 인하 기대감을 선반영한 데다 금융 당 금리는 낮췄지만 정부 재정 '바닥'… 내수회복에 쓸 예산이 없다"},
 {'url': 'https://www.chosun.com/site/data/html_dir/2011/11/10/2011111001106.html',
  'content': '영화 타짜의 정마담이 현실로10억대 사기 도박단 검거 미인계 이용 40~70대男 5명에게 10억원 챙겨 ... 오후 11시 고스톱이 끝날 때까지 a씨가 잃은 돈은 9000만원이었다. 이후 a씨는 네 차례 더 고스톱을 쳤고, 총 5억 3000여만원을 잃었다. 사기도박단에 걸린 것이었다.'},
 {'url': 'https://www.etoday.co.kr/news/view/1760604',
  'content': '정마담, \'연예계 포주\' 의혹 받는 그 정체는?…기획사와 친분 깊다는 이야기도 정마담 누구길래 정마담, 연예 기획사와 친분도? \'연예계 포주\' 의혹을 받고 있는 정마담의 정체에 대중의 시선이 집중되고 있다. 특히 ㄱ씨는 "식사에 참석한 여성 중 일부가 \'정마담\'이라 불리는 유흥업소 관계자를 통해 동원됐다"라고 주장했다. 이에 \'정마담\'이 누구인지 많은 이들이 궁금해하고 있는 상황. 다만 제보자의 주장에 따르면 정마담은 YG 엔터테인먼트 측과 깊은 친분을 유지하고 있는 것으로 전해졌다. 유흥업소 관계자라는 정마담이 연예계까지 손을 뻗고 있는 것. 관련 뉴스 주요 뉴스 많이 본 뉴스 문화·라이프 최신 뉴스 마켓 뉴스 (주)이투데이 (제호 : 이투데이) ㅣ 서울시 강남구 강남대로 556 이투데이빌딩 ㅣ ☎ 02) 799-2600 ㅣ 보도자료 및 기

In [32]:
from typing import List

from typing_extensions import TypedDict

# 사전 세팅 단계
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether to add search
        documents: list of documents
    """

    question: str
    generation: str
    web_search: str
    documents: List[str]

In [33]:
from langchain.schema import Document

def retrieve(state):
    """
    Retrieve documents from the CSV context.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    
    print("---RETRIEVE FROM CSV---")
    question = state["question"]

    # CSV에서 문서 검색
    documents = retriever.invoke(question)
    
    # 상태에 source를 'CSV'로 설정
    return {"documents": documents, "question": question, "source": "CSV"}


def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results and source as 'web'
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search 실행
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    
    # 문서에 웹 검색 결과 추가 및 source를 'web'으로 설정
    documents.append(web_results)

    return {"documents": documents, "question": question, "source": "web"}



### Edges


def generate_answer(state):
    """
    Generate answer based on whether relevant movie exists in the context.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE ANSWER---")
    question = state["question"]
    documents = state["documents"]
    source = state["source"]  # 문서의 출처 확인

    # 기본 답변 생성 (영화 정보가 있는 경우)
    generation = rag_chain.invoke({"context": documents, "question": question})
    
    # CSV에서 찾은 경우
    if source == "CSV":
        generation += "\n해당 영화 및 사건과 관련된 정보를 POV Timeline에서 확인하실 수 있습니다."
    
    # 웹 검색을 통해 찾은 경우
    elif source == "web":
        generation += "\n죄송합니다. POV Timeline은 해당 정보를 갖고 있지 않아, 관련 영화를 말씀드렸습니다. 해당 영화가 궁금하시다면 웹 검색을 추천드립니다."

    # 문서가 아예 없으면
    if not documents:
        generation = "죄송합니다. 관련 영화가 없습니다. 다른 영화를 추천 받으시겠어요?"

    return {"documents": documents, "question": question, "generation": generation}


In [34]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query
workflow.add_node("web_search_node", web_search)  # web search

# Build graph
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "web_search_node")
workflow.add_edge("web_search_node", "generate")
workflow.add_edge("generate", END)

# Compile
app = workflow.compile()

In [35]:
from pprint import pprint

# Run
inputs = {"question": "윤동주 관련 영화 알려줘"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE FROM CSV---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---
"Node 'grade_documents':"
'\n---\n'
---TRANSFORM QUERY---
"Node 'transform_query':"
'\n---\n'
---WEB SEARCH---
"Node 'web_search_node':"
'\n---\n'
---GENERATE---
"Node 'generate':"
'\n---\n'
('윤동주에 관한 추천 영화는 "동주"입니다. \n'
 '\n'
 '이 영화는 2016년에 개봉하였으며, 한국의 시인이자 독립운동가인 윤동주와 그의 사촌 송몽규의 이야기를 다루고 있습니다. 윤동주는 일제 '
 '강점기 시대에 한국 독립운동을 이끌었던 인물로, 그의 삶과 업적을 통해 당시의 아픔과 저항의 메시지를 전달합니다. \n'
 '\n'
 '영화는 박정민과 강하늘이 출연하며, 이준익 감독이 메가폰을 잡았습니다. 흑백으로 촬영된 이 작품은 비극적이고 슬픈 분위기를 자아내며, '
 '윤동주의 시인으로서의 고뇌와 저항의 의지를 잘 표현하고 있습니다.\n'
 '\n'
 '해당 영화 및 사건과 관련된 정보를 POV Timeline에서 확인하실 수 있습니다.')


In [36]:
# Run
inputs = {"question": "박정희 대통령 시기를 다룬 한국 영화 추천해줘"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE FROM CSV---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
"Node 'generate':"
'\n---\n'
('박정희 대통령 시기를 다룬 한국 영화로는 **"킹메이커"**가 있습니다. 이 영화는 1971년 제7대 대통령 선거를 배경으로 하며, '
 '박정희, 김대중, 김영삼 등의 인물들이 등장합니다. 정치와 느와르 장르로, 이선균과 설경구가 출연하며 변성현 감독이 연출했습니다. 영화는 '
 '2022년 1월 26일에 개봉되었습니다.\n'
 '\n'
 '또한, **"효자동 이발사"**라는 영화도 추천할 수 있습니다. 이 영화는 1960년대의 부정선거와 4.19 혁명, 그리고 5.16 '
 '군사정변을 배경으로 하며, 송강호가 주연을 맡았습니다. 2004년 5월 5일에 개봉된 이 영화는 역사 속의 개인을 다루고 있습니다.\n'
 '\n'
 '해당 영화 및 사건과 관련된 정보를 POV Timeline에서 확인하실 수 있습니다.')


In [37]:
# Run
inputs = {"question": "장면 정부 시기를 다룬 한국 영화 추천해줘"}
for output in app.stream(inputs):
    for key, value in output.items():
        # Node
        pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

---RETRIEVE FROM CSV---
"Node 'retrieve':"
'\n---\n'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grade_documents':"
'\n---\n'
---GENERATE---
"Node 'generate':"
'\n---\n'
('장면 정부 시기를 다룬 한국 영화로는 **"킹메이커"**를 추천합니다. \n'
 '\n'
 '이 영화는 1971년 제7대 대통령 선거를 배경으로 하며, 박정희, 김대중, 김영삼 등 주요 인물들이 등장합니다. 정치와 느와르 장르를 '
 '결합한 이 작품은 이선균과 설경구가 주연을 맡아 씁쓸한 정치적 상황을 그려냅니다. \n'
 '\n'
 '또한, **"말모이"**도 추천할 수 있습니다. 이 영화는 1942년 조선어학회 사건을 다루고 있으며, 한국어와 문화의 중요성을 강조하는 '
 '내용입니다.\n'
 '\n'
 '해당 영화 및 사건과 관련된 정보를 POV Timeline에서 확인하실 수 있습니다.')
